In [ ]:
# imports
from subprocess import check_output
import pandas as pd
import plotly.express as px
import plotly
import re
import nbformat
import time

In [ ]:
# run code for each p value repeat times from 0.1 to 0.9
question_four_df = pd.DataFrame(columns = ['Density', 'Solvability'])
repeat = 100

for i in range(10):
    count = 0
    p = 0.1 * i
    for x in range(repeat):
        command = 'python main.py -d 101 -a a_star -p {}'.format(p)
        out = str(check_output(command, cwd='/Users/advithchegu/Desktop/Rutgers/Fall 2021/AI/Assignment 1', shell=True))
        if 'No Path Found' in out:
            continue
        else:
            count += 1
    count /= repeat
    question_four_df = question_four_df.append([{'Density': p, 'Solvability': count}], ignore_index=True)

print(question_four_df)

In [ ]:
# plot the scores array
fig = px.line(question_four_df, x="Density", y="Solvability", title='Plot Density vs Solvability', text="Solvability")
fig.update_traces(textposition="bottom left")
fig.show()

plotly.io.write_image(fig, file="./graphs/question_4.png", scale=5)

In [81]:
# measure heuristic efficiency
times_df = pd.DataFrame(columns = ['Time', 'Heuristic', 'Density', 'Trajectory'])

# 6,100

for i in range (100):
    for x in range(4):

        p = 0.1*x

        command = 'python main.py -d 101 -a repeated_a_star -m combined -p ' + str(p)
        out = str(check_output(command, cwd='/Users/advithchegu/Desktop/Rutgers/Fall 2021/AI/Assignment 1', shell=True))
        
        while 'No Path Found' in out:
            out = str(check_output(command, cwd='/Users/advithchegu/Desktop/Rutgers/Fall 2021/AI/Assignment 1', shell=True))
        
        overall_arr = out.lstrip("b'").rstrip("'").replace("\\n", "").split(',')

        for i,r in enumerate(overall_arr):
            res = r.strip("'").split(" ")
            if i == 0:
                times_df = times_df.append([{'Time': float(res[0]), 'Heuristic': 'Chebyshev', 'Density': p, 'Trajectory': int(res[1])}])
            if i == 1:
                times_df = times_df.append([{'Time': float(res[0]), 'Heuristic': 'Manhattan', 'Density': p, 'Trajectory': int(res[1])}])
            if i == 2:
                times_df = times_df.append([{'Time': float(res[0]), 'Heuristic': 'Euclidian', 'Density': p, 'Trajectory': int(res[1])}])
            if i == 3:
                times_df = times_df.append([{'Time': float(res[0]), 'Heuristic': 'Weighted Manhattan', 'Density': p, 'Trajectory': int(res[1])}])
            if i == 4:
                times_df = times_df.append([{'Time': float(res[0]), 'Heuristic': 'Combined Chebyshev Manhattan', 'Density': p, 'Trajectory': int(res[1])}])
            if i == 5:
                times_df = times_df.append([{'Time': float(res[0]), 'Heuristic': 'Combined Chebyshev Euclidian', 'Density': p, 'Trajectory': int(res[1])}])
            if i == 6:
                times_df = times_df.append([{'Time': float(res[0]), 'Heuristic': 'Combined Manhattan Euclidian', 'Density': p, 'Trajectory': int(res[1])}])
        
        # times = re.search(r"b'times: \[\d+", out).group()
        # trajectories = int(re.search(r"trajectories: \[\d+", out).group().lstrip("Completed in "))
        # list(map(float, formatted))

        # , {'Time': formatted[1], 'Heuristic': 'Manhattan'}, {'Time': formatted[2], 'Heuristic': 'Euclidian'}], ignore_index=True)

        # times_df = times_df.append([{'Time': formatted[0], 'Heuristic': 'Chebyshev'}, {'Time': formatted[1], 'Heuristic': 'Manhattan'}, {'Time': formatted[2], 'Heuristic': 'Euclidian'}], ignore_index=True)

        # print(times_df)


print(times_df)

In [ ]:
# plot the box plots
fig = px.box(times_df, x="Heuristic", y="Time", title="Heuristics vs Time")
fig.show()
plotly.io.write_image(fig, file="./graphs/question_5.jpg", scale=5)

In [ ]:
question_seven_df = pd.DataFrame(columns = ['Density', 'Trajectory', 'Discovered Shortest Path', 'Final Shortest Path', 'Processed Cells', 'Trial'])

for i in range(100):
    for x in range(12):
        p = 0.03*x
        command = 'python main.py -d 101 -a repeated_a_star -m all -p ' + str(p)

        while True:
            out = str(check_output(command, cwd='/Users/advithchegu/Desktop/Rutgers/Fall 2021/AI/Assignment 1', shell=True))
            trajectory = int(re.search(r"Trajectory Length: \d+", out).group().lstrip("Trajectory Length: "))
            shortest_path = int(re.search(r"Shortest Path: \d+", out).group().lstrip("Shortest Path: "))
            full_world_shortest_path = int(re.search(r"Full Gridworld Path: \d+", out).group().lstrip("Full Gridworld Path: "))
            processed_cells = int(re.search(r"Processed \d+", out).group().lstrip("Processed "))

            if shortest_path > 0:
                break
        
        question_seven_df = question_seven_df.append([{'Density': p, 'Trajectory': trajectory, 'Discovered Shortest Path': shortest_path, 'Final Shortest Path': full_world_shortest_path, 'Processed Cells': processed_cells, 'Trial': i+1}])
# print(question_six_df)

In [ ]:
grouped_df = question_seven_df.groupby('Density')
avg_seven_df = grouped_df.mean()
avg_seven_df = avg_seven_df.reset_index()

In [ ]:
# do some calculations
avg_seven_df['Trajectory / Shortest Path (Disc.)'] = round(avg_seven_df['Trajectory'] / avg_seven_df['Discovered Shortest Path'], 3)
avg_seven_df['Shortest Path (Disc.) / Shortest Path (Final)'] = round(avg_seven_df['Discovered Shortest Path'] / avg_seven_df['Final Shortest Path'], 3)

In [ ]:

fig1 = px.line(avg_seven_df, x="Density", y="Trajectory", title='Density vs Average Trajectory Length (Reduced Field)', text='Trajectory')
fig1.update_xaxes(dtick=0.03)
fig1.update_traces(textposition="bottom right")
fig1.show()

plotly.io.write_image(fig1, file="./graphs/question_7_1.png", scale=5)

In [ ]:
fig2 = px.line(avg_seven_df, x="Density", y="Trajectory / Shortest Path (Disc.)", title='Density vs Average Shortest Path Lengths (Reduced Field)', text='Trajectory / Shortest Path (Disc.)')
fig2.update_xaxes(dtick=0.03)
fig2.update_traces(textposition="bottom right")
fig2.show()

plotly.io.write_image(fig2, file="./graphs/question_7_2.png", scale=5)

In [ ]:
fig3 = px.line(avg_seven_df, x="Density", y="Shortest Path (Disc.) / Shortest Path (Final)", title='Density vs Average Shortest Path Lengths (Reduced Field)', text='Shortest Path (Disc.) / Shortest Path (Final)')
fig3.update_xaxes(dtick=0.03)
fig3.update_traces(textposition="bottom right")
fig3.show()

plotly.io.write_image(fig3, file="./graphs/question_7_3.png", scale=5)

In [ ]:
fig4 = px.line(avg_seven_df, x="Density", y="Processed Cells", title='Density vs Processed Cells', text='Processed Cells')
fig4.update_xaxes(dtick=0.03)
fig4.update_traces(textposition="bottom right")
fig4.show()

plotly.io.write_image(fig4, file="./graphs/question_7_4.png", scale=5)

In [ ]:
question_seven_df.to_csv('misc/question_7.csv')
avg_seven_df.to_csv('misc/question_7_avg.csv')